In [1]:
pip install -qU "langchain[google-genai]"


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import os
if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [17]:
from langchain_core.messages import HumanMessage,SystemMessage
message=[
    SystemMessage('Give result based on tool call'),
    HumanMessage("Sum of 10+10")

]
model.invoke(message)

AIMessage(content='```tool_code\nprint(10+10)\n```', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--19d2d3e1-5974-41c6-94cf-951115188338-0', usage_metadata={'input_tokens': 14, 'output_tokens': 15, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}})

In [18]:
from langchain_core.prompts import ChatPromptTemplate
system_template="Give result based on tool call"
prompt_template=ChatPromptTemplate.from_messages([("system",system_template),("user","{text}")])

In [19]:
prompt=prompt_template.invoke({"text":"Bangalore"})

In [20]:
print(model.invoke(prompt))

content='```tool_code\nprint(google_search.search(queries=["Bangalore"]))\n```' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--53777729-66cb-4ace-9ef6-e9cca4ece1c5-0' usage_metadata={'input_tokens': 7, 'output_tokens': 20, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}}


In [21]:
from langchain_core.tools import tool

@tool
def get_add(a:int,b:int)->int:
    """"Add the two numbers and return result"""
    return a+b


@tool
def get_sub(a:int,b:int)->int:
    """"Subtract  two numbers and return result"""
    return a-b

model_with_tool=model.bind_tools([get_add,get_sub])

result=model_with_tool.invoke('what is the add of 10 and 10')
print(result.content)

In [27]:
result=model_with_tool.invoke('result of 10+10')
print(result)

content='' additional_kwargs={'function_call': {'name': 'get_add', 'arguments': '{"a": 10.0, "b": 10.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--906da79e-8a4f-417f-aa8c-be2e55b73965-0' tool_calls=[{'name': 'get_add', 'args': {'a': 10.0, 'b': 10.0}, 'id': 'b7324b41-06b7-4c7d-87b6-cf5f806f4817', 'type': 'tool_call'}] usage_metadata={'input_tokens': 44, 'output_tokens': 7, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}}


In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash")
result=llm.invoke("What is the capital of IL?")
print(result.content)

The capital of Illinois is **Springfield**.


In [35]:
message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "What's in this image?",
        },
        {"type": "image_url", "image_url": "https://picsum.photos/seed/picsum/200/300"},
    ]
)
result = llm.invoke([message])
print(result.content)

Here's what I can see in the image:

*   **Landscape:** The image depicts a mountain landscape.
*   **Mountains:** There is a prominent snow-covered mountain peak visible on the right side of the image.
*   **Sky:** The sky takes up a significant portion of the image, featuring soft, pinkish-hued clouds.
*   **Snow:** There appears to be a large, snow-covered field or slope in the foreground.
*   **Lighting:** The lighting suggests it might be either sunrise or sunset, given the soft, warm tones in the sky.


In [36]:
from pydantic import BaseModel, Field
class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    answer: str = Field(description="The answer to the user's question")
    followup_question: str = Field(description="A followup question the user could ask")


In [38]:

# Bind responseformatter schema as a tool to the model
model_with_tools = model.bind_tools([ResponseFormatter])
# Invoke the model
ai_msg = model_with_tools.invoke("What is the powerhouse of the cell?")


In [41]:
# Get the tool call arguments
ai_msg.tool_calls[0]["args"]
{'answer': "The powerhouse of the cell is the mitochondrion. Mitochondria are organelles that generate most of the cell's supply of adenosine triphosphate (ATP), which is used as a source of chemical energy.",
 'followup_question': 'What is the function of ATP in the cell?'}
# Parse the dictionary into a pydantic object
pydantic_object = ResponseFormatter.model_validate(ai_msg.tool_calls[0]["args"])

In [47]:

model=ChatGoogleGenerativeAI(model="gemini-2.0-flash").with_structured_output(method="json_mode",schema=ResponseFormatter)
ai_msg = model.invoke("Return a JSON object with key 'random_ints' and a value of 10 random ints in [100-199]")
ai_msg


ResponseFormatter(answer='Here are 10 random integers between 100 and 199: [101, 123, 145, 167, 189, 112, 134, 156, 178, 190]', followup_question='Would you like a different set of random integers or a different range?')